In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
data_path='training_set_filtered.csv'
data=pd.read_csv(data_path)

In [3]:
data.shape

(8152372, 8)

In [4]:
data =data [['ExecutionTime','high','low','close','volume']] 
data.head()

,ExecutionTime,high,low,close,volume
0,2021-01-07 20:15:00+01:00,57.28,55.2,57.28,1.125
1,2021-01-07 20:30:00+01:00,61.90,59.7,61.30,5.150
2,2021-01-07 20:45:00+01:00,61.90,59.7,61.30,0.000
3,2021-01-07 21:00:00+01:00,59.94,56.1,56.10,0.200
4,2021-01-07 21:15:00+01:00,63.75,61.0,63.47,2.525


In [5]:
data_new=data[['high','low','close','volume']]
data_new.head()

,high,low,close,volume
0,57.28,55.2,57.28,1.125
1,61.90,59.7,61.30,5.150
2,61.90,59.7,61.30,0.000
3,59.94,56.1,56.10,0.200
4,63.75,61.0,63.47,2.525


In [6]:
data_values = data_new.to_numpy(dtype=np.float32)

In [7]:
context_length = 34  # The number of past steps to consider
prediction_length = 10  # Number of future steps to predict
# Create a sliding window over the time series data to extract past values
sequence_length = context_length# Total past sequence length
# Split the data into sliding windows
past_values = []
future_values = []
# Sliding window with a step size of sequence_length
for i in range(0, len(data_values) - sequence_length - prediction_length, sequence_length+prediction_length):
    past_window = data_values[i : i + sequence_length]  # Past values for input
    future_window = data_values[i + sequence_length : i + sequence_length + prediction_length]  # Future values
    
    past_values.append(past_window)
    future_values.append(future_window)

# Convert lists to PyTorch tensors
past_values_tensor = torch.tensor(np.array(past_values), dtype=torch.float32)  # Shape: (num_windows, sequence_length, num_features)
future_values_tensor = torch.tensor(np.array(future_values), dtype=torch.float32)  # Shape: (num_windows, prediction_length, num_features)

# Check the shapes
print(f"Past values tensor shape: {past_values_tensor.shape}")
print(f"Future values tensor shape: {future_values_tensor.shape}")


Past values tensor shape: torch.Size([185281, 34, 4])
Future values tensor shape: torch.Size([185281, 10, 4])


In [8]:
data['ExecutionTime'] = pd.to_datetime(data['ExecutionTime']) 

C:\Users\wuzhu\AppData\Local\Temp\ipykernel_28216\2844849870.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  data['ExecutionTime'] = pd.to_datetime(data['ExecutionTime'])


In [9]:
def transform_execution_time(timestamp):
    hour_decimal = timestamp.hour + timestamp.minute / 60.0
    year = timestamp.year
    month_day_decimal = timestamp.month + timestamp.day / 100.0  # Combine month and day as a decimal
    return [hour_decimal,month_day_decimal,year]

# Apply the transformation to the 'ExecutionTime' column
time_features = data['ExecutionTime'].apply(transform_execution_time)

In [10]:
time_features = time_features.apply(lambda x: [float(i) for i in x])

# Check the result
time_features.head()

0    [20.25, 1.07, 2021.0]
1     [20.5, 1.07, 2021.0]
2    [20.75, 1.07, 2021.0]
3     [21.0, 1.07, 2021.0]
4    [21.25, 1.07, 2021.0]
Name: ExecutionTime, dtype: object

In [11]:
# Convert the series to a numpy array of floats
time_features = np.array(time_features.tolist(), dtype=np.float32)

In [12]:
# Lists to store past time feature windows and future time feature windows
past_time_features = []
future_time_features = []

# Sliding window with a step size of sequence_length
for i in range(0, len(time_features) - sequence_length - prediction_length, sequence_length + prediction_length):
    past_window = time_features[i : i + sequence_length]  # Past time features for input
    future_window = time_features[i + sequence_length : i + sequence_length + prediction_length]  # Future time features
    
    past_time_features.append(past_window)
    future_time_features.append(future_window)

# Convert lists to PyTorch tensors
past_time_features_tensor = torch.tensor(np.array(past_time_features), dtype=torch.float32)  # Shape: (num_windows, sequence_length, num_features)
future_time_features_tensor = torch.tensor(np.array(future_time_features), dtype=torch.float32)  # Shape: (num_windows, prediction_length, num_features)

# Check the shapes
print(f"Past time features tensor shape: {past_time_features_tensor.shape}")
print(f"Future time features tensor shape: {future_time_features_tensor.shape}")

Past time features tensor shape: torch.Size([185281, 34, 3])
Future time features tensor shape: torch.Size([185281, 10, 3])


In [13]:
data_path='testing_set_filtered.csv'
data2=pd.read_csv(data_path)
data2.head()

,Unnamed: 0,ExecutionTime,ID,high,low,close,volume,TimeDiff
0,463,2024-01-11 16:15:00+01:00,Fri00Q1,99.94,89.94,93.50,6.600,NaN
1,464,2024-01-11 16:30:00+01:00,Fri00Q1,99.94,89.94,93.50,0.000,0 days 00:15:00
2,465,2024-01-11 16:45:00+01:00,Fri00Q1,94.75,94.20,94.75,1.225,0 days 00:15:00
3,466,2024-01-11 17:00:00+01:00,Fri00Q1,96.25,96.25,96.25,0.125,0 days 00:15:00
4,467,2024-01-11 17:15:00+01:00,Fri00Q1,93.50,93.50,93.50,0.025,0 days 00:15:00


In [14]:
ID = data2[['ID']]

In [15]:
data2_new=data2[['high','low','close','volume']]
data2_new.head()

,high,low,close,volume
0,99.94,89.94,93.50,6.600
1,99.94,89.94,93.50,0.000
2,94.75,94.20,94.75,1.225
3,96.25,96.25,96.25,0.125
4,93.50,93.50,93.50,0.025


In [16]:
# Convert DataFrame to NumPy array
data_values = data2_new.to_numpy(dtype=np.float32)

In [17]:
context_length = 34  # The number of past steps to consider
prediction_length = 10  # Number of future steps to predict
# Create a sliding window over the time series data to extract past values
sequence_length = context_length# Total past sequence length
# Split the data into sliding windows
past_values = []
future_values = []
ID_list=[]
# Sliding window with a step size of sequence_length
for i in range(0, len(data_values) - sequence_length - prediction_length,sequence_length + prediction_length):
    past_window = data_values[i : i + sequence_length]  # Past values for input
    future_window = data_values[i + sequence_length : i + sequence_length + prediction_length]  # Future values
    ID_window = ID[i + sequence_length : i + sequence_length + prediction_length]
    past_values.append(past_window)
    future_values.append(future_window)
    ID_list.append(ID_window)
# Convert lists to PyTorch tensors
past_values_tensor_test = torch.tensor(np.array(past_values), dtype=torch.float32)  # Shape: (num_windows, sequence_length, num_features)
future_values_tensor_test = torch.tensor(np.array(future_values), dtype=torch.float32)  # Shape: (num_windows, prediction_length, num_features)
ID_array = np.array(ID_list)
# Check the shapes
print(f"test past values tensor shape: {past_values_tensor_test.shape}")
print(f"test future values tensor shape: {future_values_tensor_test.shape}")


test past values tensor shape: torch.Size([38949, 34, 4])
test future values tensor shape: torch.Size([38949, 10, 4])


In [18]:
data2['ExecutionTime'] = pd.to_datetime(data2['ExecutionTime']) 

C:\Users\wuzhu\AppData\Local\Temp\ipykernel_28216\1534058427.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  data2['ExecutionTime'] = pd.to_datetime(data2['ExecutionTime'])


In [19]:
def transform_execution_time(timestamp):
    hour_decimal = timestamp.hour + timestamp.minute / 60.0
    year = timestamp.year
    month_day_decimal = timestamp.month + timestamp.day / 100.0  # Combine month and day as a decimal
    return [hour_decimal,month_day_decimal,year]

# Apply the transformation to the 'ExecutionTime' column
time_features2 = data2['ExecutionTime'].apply(transform_execution_time)

In [20]:
time_features2 = time_features2.apply(lambda x: [float(i) for i in x])
# Check the result
time_features2.head()

0    [16.25, 1.11, 2024.0]
1     [16.5, 1.11, 2024.0]
2    [16.75, 1.11, 2024.0]
3     [17.0, 1.11, 2024.0]
4    [17.25, 1.11, 2024.0]
Name: ExecutionTime, dtype: object

In [21]:
# Convert the series to a numpy array of floats
time_features2 = np.array(time_features2.tolist(), dtype=np.float32)

In [22]:
# Lists to store past time feature windows and future time feature windows
past_time_features2 = []
future_time_features2 = []

# Sliding window with a step size of sequence_length
for i in range(0, len(time_features2) - sequence_length - prediction_length, sequence_length + prediction_length):
    past_window = time_features2[i : i + sequence_length]  # Past time features for input
    future_window = time_features2[i + sequence_length : i + sequence_length + prediction_length]  # Future time features
    
    past_time_features2.append(past_window)
    future_time_features2.append(future_window)

# Convert lists to PyTorch tensors
past_time_features_tensor_test = torch.tensor(np.array(past_time_features2), dtype=torch.float32)  
future_time_features_tensor_test = torch.tensor(np.array(future_time_features2), dtype=torch.float32)

# Check the shapes
print(f"test past time features tensor shape: {past_time_features_tensor_test.shape}")
print(f"test future time features tensor shape: {future_time_features_tensor_test.shape}")

test past time features tensor shape: torch.Size([38949, 34, 3])
test future time features tensor shape: torch.Size([38949, 10, 3])


In [23]:
torch.save(past_time_features_tensor, 'past_time_features_tensor.pt')
torch.save(future_time_features_tensor, 'future_time_features_tensor.pt')
torch.save(past_values_tensor,'past_value_tensor.pt')
torch.save(future_values_tensor, 'future_value_tensor.pt')
torch.save(past_time_features_tensor_test, 'test_past_time_features_tensor.pt')
torch.save(future_time_features_tensor_test, 'test_future_time_features_tensor.pt')
torch.save(past_values_tensor_test,'test_past_value_tensor.pt')
torch.save(future_values_tensor_test, 'test_future_value_tensor.pt')

In [24]:
ID_array.shape

(38949, 10, 1)

In [25]:
import numpy as np

np.save('ID', ID_array)